# Regularização L1 L2 VS Dropout

## Regressão com o dataset MPG

In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
# Desativa os warnings
warnings.filterwarnings('ignore')

In [3]:
# Carregando o dataset
dataset = pd.read_csv('data-mpg/auto-mpg.csv', na_values=['NA', '?'])

In [4]:
# Verificando a quantidade de valores nulos no dataset
dataset.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [5]:
# Exibindo os dados
dataset.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [6]:
# Funções auxiliares

# Converte todos os valores faltantes na coluna especificada para a mediana
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Converte um dataframe Pandas para as entradas x, y que o TensorFlow precisa
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # Descobre o tipo da coluna de destino. 
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encoding para int. TensorFlow gosta de 32 bits.
    if target_type in (np.int64, np.int32):
        # Classificação
        dummies = pd.get_dummies(df[target])
        return df[result].to_numpy(dtype=np.float32), dummies.to_numpy(dtype=np.float32)
    else:
        # Regressão
        return df[result].to_numpy(dtype=np.float32), df[target].to_numpy(dtype=np.float32)

In [7]:
# Pré-processamento
dataset.drop('name', 1, inplace = True)
missing_median(dataset, 'horsepower')
X, Y = to_xy(dataset, 'mpg')

In [8]:
# Dividindo os dados de treino e validação
x_train, x_validation, y_train, y_validation = train_test_split(X, Y, test_size = 0.25)

In [9]:
# Shape dos dados de treino e validação
print("X train: ", x_train.shape)
print("X validation", x_validation.shape)
print("Y train: ", y_train.shape)
print("Y validation: ", y_validation.shape)

X train:  (298, 7)
X validation (100, 7)
Y train:  (298,)
Y validation:  (100,)


In [10]:
# Construindo o modelo
model1 = Sequential()
model1.add(Dense(input_dim = x_train.shape[1],
                 units = 128,
                 kernel_initializer = 'normal',
                 activation = 'relu'))

model1.add(Dense(input_dim = 128,
                 units = 64,
                 kernel_initializer = 'normal',
                 activation = 'relu'))

model1.add(Dense(input_dim = 64, 
                 units = 10,
                 kernel_initializer = 'normal',
                 activation = 'relu',
                 kernel_regularizer = regularizers.l2(0.01),
                 activity_regularizer = regularizers.l1(0.01)))

model1.add(Dense(input_dim = 64,
                 units = 1,
                 kernel_initializer = 'normal'))

model1.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [11]:
# Verificando a arquitetura da rede
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 9,941
Trainable params: 9,941
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Definindo as callbacks
monitor = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 10, verbose = 3, mode = 'auto')

plateau = ReduceLROnPlateau(monitor = 'loss',
                            factor = 0.01,
                            patience = 5,
                            mode = 'min',
                            min_lr = 0.000001,
                            verbose = 3)

In [13]:
%time
# Treinando o modelo

model1.fit(x_train,
           y_train,
           validation_data=(x_validation, y_validation),
           callbacks = [monitor, plateau],
           batch_size = 64,
           epochs = 1000)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
Epoch 1/1000
5/5 [==============================] - 0s 33ms/step - loss: 481.0553 - val_loss: 349.7662 - lr: 0.0010
Epoch 2/1000
5/5 [==============================] - 0s 11ms/step - loss: 285.6888 - val_loss: 184.5218 - lr: 0.0010
Epoch 3/1000
5/5 [==============================] - 0s 10ms/step - loss: 195.8353 - val_loss: 194.4182 - lr: 0.0010
Epoch 4/1000
5/5 [==============================] - 0s 10ms/step - loss: 233.1296 - val_loss: 175.5418 - lr: 0.0010
Epoch 5/1000
5/5 [==============================] - 0s 10ms/step - loss: 188.1895 - val_loss: 175.4168 - lr: 0.0010
Epoch 6/1000
5/5 [==============================] - 0s 13ms/step - loss: 197.0308 - val_loss: 189.2932 - lr: 0.0010
Epoch 7/1000
5/5 [==============================] - 0s 12ms/step - loss: 200.1477 - val_loss: 176.0066 - lr: 0.0010
Epoch 8/1000
5/5 [==============================] - 0s 12ms/step - loss: 188.8386 - val_loss: 165.4679 - lr: 0.0010
Epoch 9/

5/5 [==============================] - 0s 10ms/step - loss: 20.4119 - val_loss: 36.2033 - lr: 0.0010
Epoch 72/1000
5/5 [==============================] - 0s 9ms/step - loss: 20.0830 - val_loss: 34.7824 - lr: 0.0010
Epoch 73/1000
5/5 [==============================] - 0s 10ms/step - loss: 19.4203 - val_loss: 34.8455 - lr: 0.0010
Epoch 74/1000
5/5 [==============================] - 0s 8ms/step - loss: 19.1709 - val_loss: 34.5965 - lr: 0.0010
Epoch 75/1000
5/5 [==============================] - 0s 9ms/step - loss: 19.1958 - val_loss: 33.1070 - lr: 0.0010
Epoch 76/1000
5/5 [==============================] - 0s 8ms/step - loss: 19.1455 - val_loss: 32.9656 - lr: 0.0010
Epoch 77/1000
5/5 [==============================] - 0s 9ms/step - loss: 18.9979 - val_loss: 36.7985 - lr: 0.0010
Epoch 78/1000
5/5 [==============================] - 0s 10ms/step - loss: 19.1696 - val_loss: 33.8108 - lr: 0.0010
Epoch 79/1000
5/5 [==============================] - 0s 11ms/step - loss: 18.6023 - val_loss: 32.41

In [14]:
# Fazendo as previsões
pred1 = model1.predict(x_validation)

# Cálcula o RSME (Root square mean error )
rmse = np.sqrt(metrics.mean_squared_error(pred1, y_validation))

# Exibe o rmse
print("RSME: ", rmse)

RSME:  4.627955


In [15]:
# Construindo o modelo
model2 = Sequential()
model2.add(Dense(input_dim = x_train.shape[1],
                 units = 128,
                 kernel_initializer = 'normal',
                 activation = 'relu'))

model2.add(Dense(input_dim = 128,
                 units = 64,
                 kernel_initializer = 'normal',
                activation = 'relu'))

model2.add(Dropout(0.25))

model2.add(Dense(input_dim = 64, 
                 units = 10,
                 kernel_initializer = 'normal',
                 activation = 'relu'))

model2.add(Dense(input_dim = 64,
                 units = 1,
                 kernel_initializer = 'normal'))

model2.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [16]:
# Exibindo a arquitetura do modelo
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 9,941
Trainable params: 9,941
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Treinando o modelo
model2.fit(x_train,
           y_train,
           validation_data = (x_validation, y_validation),
           callbacks = [monitor, plateau],
           batch_size = 64,
           epochs = 1000)

Epoch 1/1000
5/5 [==============================] - 0s 19ms/step - loss: 569.7275 - val_loss: 453.0809 - lr: 0.0010
Epoch 2/1000
5/5 [==============================] - 0s 10ms/step - loss: 393.0148 - val_loss: 286.2719 - lr: 0.0010
Epoch 3/1000
5/5 [==============================] - 0s 10ms/step - loss: 244.0111 - val_loss: 166.9273 - lr: 0.0010
Epoch 4/1000
5/5 [==============================] - 0s 11ms/step - loss: 205.0907 - val_loss: 181.2050 - lr: 0.0010
Epoch 5/1000
5/5 [==============================] - 0s 11ms/step - loss: 223.9027 - val_loss: 163.4240 - lr: 0.0010
Epoch 6/1000
5/5 [==============================] - 0s 10ms/step - loss: 199.9027 - val_loss: 177.5143 - lr: 0.0010
Epoch 7/1000
5/5 [==============================] - 0s 9ms/step - loss: 200.0428 - val_loss: 184.6081 - lr: 0.0010
Epoch 8/1000
5/5 [==============================] - 0s 10ms/step - loss: 195.8173 - val_loss: 172.9558 - lr: 0.0010
Epoch 9/1000
5/5 [==============================] - 0s 10ms/step - loss: 

Epoch 72/1000
5/5 [==============================] - 0s 10ms/step - loss: 27.6934 - val_loss: 35.6347 - lr: 0.0010
Epoch 73/1000
5/5 [==============================] - 0s 8ms/step - loss: 30.6068 - val_loss: 34.1787 - lr: 0.0010
Epoch 74/1000
5/5 [==============================] - 0s 8ms/step - loss: 29.7984 - val_loss: 32.9254 - lr: 0.0010
Epoch 75/1000
5/5 [==============================] - 0s 8ms/step - loss: 28.3881 - val_loss: 32.9264 - lr: 0.0010
Epoch 76/1000
5/5 [==============================] - 0s 9ms/step - loss: 28.4499 - val_loss: 34.4063 - lr: 0.0010
Epoch 77/1000
1/5 [=====>........................] - ETA: 0s - loss: 20.7489
Epoch 00077: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
5/5 [==============================] - 0s 8ms/step - loss: 30.9099 - val_loss: 35.8365 - lr: 0.0010
Epoch 78/1000
5/5 [==============================] - 0s 8ms/step - loss: 30.5308 - val_loss: 35.7161 - lr: 1.0000e-05
Epoch 79/1000
5/5 [==============================] - 

In [18]:
# Fazendo as previsões
pred2 = model2.predict(x_validation)

# Cálcula o RSME (Root square mean error )
rmse = np.sqrt(metrics.mean_squared_error(pred2, y_validation))

# Exibe o rmse
print("RSME: ", rmse)

RSME:  5.7865973


## Conclusão

O modelo treinado utilizando a técnica de regularização L1 L2 teve melhor resultado quando comparado ao modelo treinado utilizando a técnica de regularização Dropout, porém, o modelo tomou mais épocas e consequentemente mais tempo do que com Dropout.